In [ ]:
import os
import re
import time
import joblib
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score

In [ ]:
pwd_path = os.path.dirname(os.path.abspath(__file__))
data = pd.read_csv(os.path.join(
    pwd_path, 'TextClassification Dataset - main (3).csv'))

Limpeza de dados e remoção de stop words

In [ ]:
def clean_url(url):
    stop_words = set(stopwords.words('portuguese'))
    cleaned_url = ' '.join(word for word in re.split(
        '[^a-zA-Z6]', url) if word not in stop_words).lower()
    cleaned_url = re.sub('www', '', cleaned_url)

    return cleaned_url

Treinamento do modelo utilizando CountVectorizer e Redes Neurais.

In [ ]:
print("I'll start the training now...")
t1 = time.time()
data['cleaned_username'] = data['username'].apply(clean_url)

train_data, test_data = train_test_split(data,
                                         test_size=0.2,
                                         random_state=42)

vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(train_data['cleaned_username'])
x_test = vectorizer.transform(test_data['cleaned_username'])

clf = MLPClassifier(hidden_layer_sizes=(350,), activation='relu',
                    solver='adam',
                    max_iter=4500, random_state=42, verbose=True)
model = clf.fit(x_train, train_data['clientes'])

t = time.time()
elapsed_time = t - t1
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)
print(f'Training completed in: {minutes}m {seconds}s \n')

y_pred = model.predict(x_test)
accuracy = accuracy_score(test_data['clientes'], y_pred)
accuracy = round(float(accuracy * 100), 4)

FILENAME = 'text_model2.pkl'
VECTORIZER_FILE = 'text_vectorizer2.pkl'
joblib.dump(model, FILENAME)
joblib.dump(vectorizer, VECTORIZER_FILE)

print(f'Accuracy of the Neural Network: {accuracy}%')
print(' ')